In [45]:
import geopandas as gpd

In [46]:
amenities_file = '/app/data/amenities'

amn = gpd.read_file(amenities_file)
amn['name'].fillna('', inplace=True)

idna = amn['id'].isna()
amn.loc[idna, 'id'] = ('2024' + amn[idna].index.astype(str)).astype(int)

futr = amn['name'].str.contains('futur', case=False)
amn.loc[~futr, 'scenario'] = 'actual'
amn.loc[ futr, 'scenario'] = 'future'

amn_cols = ['id', 'Category', 'scenario', 'geometry']
amn = amn[amn_cols]
amn
# amn[amn['Category']=='Cultura']

,id,Category,scenario,geometry
0,1280585100,Entretenimiento,actual,POINT Z (673882.866 5921852.428 0.000)
1,1280906720,Comida para servir,actual,POINT Z (673587.739 5921888.825 0.000)
2,1280963868,Entretenimiento,actual,POINT Z (673615.644 5920946.613 0.000)
3,1507046243,Servicios,actual,POINT Z (673665.025 5922374.360 0.000)
4,1507046253,Servicios,actual,POINT Z (673610.110 5922345.688 0.000)
...,...,...,...,...
882,2024882,Cultura,actual,POINT Z (673801.359 5922528.531 0.000)
883,2024883,Cultura,actual,POINT Z (675685.196 5923795.888 0.000)
884,2024884,Cultura,actual,POINT Z (673878.220 5921788.358 0.000)
885,2024885,Cultura,actual,POINT Z (674859.165 5922484.875 0.000)


In [47]:
amn = gpd.GeoDataFrame(data=amn.drop(columns=['geometry']), geometry=amn['geometry'])

In [48]:
amn['id'] = amn['id'].astype(str)

In [29]:
amn.loc[amn['scenario']=='actual', ['id', 'Category', 'geometry']].to_parquet('/app/assets/amenities/actual.parquet')
amn.loc[:, ['id', 'Category', 'geometry']].to_parquet('/app/assets/amenities/future.parquet')

In [57]:
amn_costanera = gpd.read_file('/app/data/amenities_costanera_v2')

In [58]:
amn_costanera['scenario'] = 'actual'

In [59]:
amn.head()

,id,Category,scenario,geometry
0,1280585100,Entretenimiento,actual,POINT Z (673882.866 5921852.428 0.000)
1,1280906720,Comida para servir,actual,POINT Z (673587.739 5921888.825 0.000)
2,1280963868,Entretenimiento,actual,POINT Z (673615.644 5920946.613 0.000)
3,1507046243,Servicios,actual,POINT Z (673665.025 5922374.360 0.000)
4,1507046253,Servicios,actual,POINT Z (673610.110 5922345.688 0.000)


In [60]:
amn_costanera = amn_costanera[['id', 'Category', 'scenario', 'geometry']]
amn_costanera.shape

(4573, 4)

In [63]:
amn_costanera.drop_duplicates(subset='geometry', inplace=True)

In [66]:
amn_costanera = amn_costanera.to_crs(amn.crs)

In [68]:
from shapely.geometry import Point

# Suponiendo que 'gdf' es tu GeoDataFrame
# Itera sobre las geometrías y crea nuevos puntos 2D
amn['geometry'] = amn['geometry'].apply(lambda geom: Point(geom.x, geom.y))

In [83]:
import pandas as pd
amn_export = pd.concat([amn, amn_costanera])
amn_export.drop_duplicates(subset=['geometry'], inplace=True)
amn_export.drop_duplicates(subset=['id'], inplace=True)

In [96]:
amn_export = gpd.GeoDataFrame(
    data=amn_export.drop(columns=['geometry']),
    geometry=amn_export['geometry'],
)

In [98]:
export_cols = ['id', 'Category', 'geometry']
amn_export.loc[amn_export['scenario']=='actual', export_cols].to_parquet('/app/assets/amenities/actual.parquet')
amn_export[export_cols].to_parquet('/app/assets/amenities/future.parquet')